In [1]:
# import mpld3
# mpld3.enable_notebook()

import matplotlib
# matplotlib.use('TkAgg')

import torch, sys
sys.path.append('..')
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from src.data.base_dataset import BasicDatasetLT

from src import options
from src.models.fc_model import BaseModelLT

from src.utils.complex_tensors import complextorch2numpy
import src.utils.visualizations as vis

import matplotlib.pyplot as plt
import glob
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots

# %matplotlib notebook
%matplotlib widget
# %matplotlib ipympl

In [2]:
# --- load model ---

# CKPT_PATH = '/Users/ranweisman/PycharmProjects/nn-ambisonics-upscaler/experiments/cloud_experiments/fc_hp_tune/fc_imagemethod_lowrank/version_467/checkpoints/epoch=99.ckpt'

# files = glob.glob('/Users/ranweisman/PycharmProjects/nn-ambisonics-upscaler/experiments/cloud_experiments/fc_hp_tune/fc_imagemethod_full/*/checkpoints/*.ckpt')
CKPT_PATH = '/Users/ranweisman/PycharmProjects/nn-ambisonics-upscaler/experiments/cloud_experiments/fc_hp_tune/fc_imagemethod_full/version_202/checkpoints/epoch=99.ckpt'
# CKPT_PATH = files[221]


checkpoint = torch.load(CKPT_PATH,map_location=torch.device('cpu'))
opts = checkpoint['hyper_parameters']['opts'].copy()
del opts['data_path'],opts['logs_path']

base_opts = options.get_default_opts(opts)
full_opts = options.prepare_opts(base_opts,opts)
full_opts = options.validate_opts(full_opts)

model = BaseModelLT(opts)
model.setup('test')
model.load_state_dict(checkpoint['state_dict'])
trainer = Trainer()
model.eval()

# # --- load data ---
#  loader = DataLoader(model.dataset_train, batch_size=model.hparams.batch_size, num_workers=model.hparams.num_workers)
#
# # --- estimate performance over all dataset ---
#  trainer.test(model,test_dataloaders=loader)

../src/options/__init__.py:85: UserWarning: GPU is not available, using CPU instead
  warnings.warn('GPU is not available, using CPU instead')


Configuration Parameters: 
experiment_name: fc_imagemethod_full
model_name: fc
rank: None
hidden_layers: 2
hidden_sizes: [3000, 3000]
residual_flag: True
residual_only: False
loss: mse
transform: None
batch_size: 10
num_workers: 8
train_val_split: [0.9, 0.1]
preload_data: True
lr: 0.0001
max_epochs: 100
gpus: 0
input_size: 512
output_shape: torch.Size([2, 49, 49])
last_layer_size: 4802
data_path: /Users/ranweisman/PycharmProjects/nn-ambisonics-upscaler/notebooks/../data/raw/image-method
logs_path: /Users/ranweisman/PycharmProjects/nn-ambisonics-upscaler/notebooks/../experiments


GPU available: False, used: False
TPU available: False, using: 0 TPU cores


BaseModelLT(
  (linears): ModuleList(
    (0): Linear(in_features=512, out_features=3000, bias=True)
    (1): Linear(in_features=3000, out_features=3000, bias=True)
    (2): Linear(in_features=3000, out_features=4802, bias=True)
  )
)

In [5]:
# --- loop - draw a single sample and plot ---

dataset = BasicDatasetLT(model.hparams.data_path, train=False)
loader = DataLoader(dataset, batch_size=1, num_workers=0, shuffle=True)

# loader = DataLoader(model.dataset_train, batch_size=1, num_workers=0, shuffle=True)
for x,y in loader:

    y_hat = model(x)

    x = complextorch2numpy(x[0])
    y = complextorch2numpy(y[0])
    y_hat = complextorch2numpy(y_hat[0])

    vis.compare_covs(x, y_hat, y)
    vis.compare_power_maps(x, y_hat, y)
    
    print(x[0])
    
    break

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 0.22660959+0.j         -0.09698426-0.02019086j -0.04476041+0.09965498j
  0.06924256+0.01552413j  0.00867056-0.01190673j -0.00907483-0.10580169j
 -0.06110267+0.03819351j -0.06272898+0.05898398j  0.02554221-0.05228046j
  0.17901121+0.07579478j  0.06194448+0.01789838j  0.02841914+0.04746556j
 -0.03256223-0.01424292j  0.01785527+0.02094028j -0.00651133-0.05367784j
 -0.00317138+0.0419485j ]
